# Assignment nº2
### Implementation and Analysis of Decision Tree Induction for Supervised Machine Learning
#### Work assembled by Alejandro Gonçalves, Francisca Mihalache and Pedro Fernandes

### Table of contents <a name="contents"></a>

1. [Introduction](#introduction)
2. [Datasets](#datasets)
   - 2.1. [Restaurant](#restaurant1)
   - 2.2. [Weather](#weather1)
   - 2.3. [Iris](#iris1)
   - 2.4. [Connect4](#connect41)
3. [ID3](#id3)
   - 3.1. [Entropy](#entropy)
   - 3.2. [Information Gain](#information_gain)
   - 3.3. [Example](#example1)
4. [ID3 Implementation](#id3-implementation)
4.1. [Restaurant](#restaurant2)
5. [Results/Predictions??](#results)
6. [Conclusions](#conclusions)

## Introduction <a name="introduction"></a>
The goal is to implement the ID3 algorithm to learn decision trees from data, ensuring the program can both build the tree and classify new instances across different datasets, without using automatic libraries. The focus is on mastering the theoretical concepts and practically applying them to solve classification tasks effectively.

### Imports

In [926]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import math
from collections import Counter

# Datasets <a name="datasets"></a>
[[go back to the top]](#contents)

In this section, we will dive into the details of our datasets, exploring the nature of the data and the predictive outcomes they are associated with.

**Note**: for those with ID's, that column is dropped, since it has no predictive value.

### Restaurant <a name="restaurant"></a>
[[go back to the topic]](#datasets)

This csv contains some examples of decision making. Given some information, the target is to decide whether the client waits or not.
Each line, each instance contains 12 attributes, split among these categories.

1. **Identification and Basic Information**:
   - ID: Personal identifier for each restaurant entry.
   - Type: Classification of the type of cuisine offered by the restaurant.

2. **Dining Options within the Restaurant**:
   - Alt: Availability of alternative dining options.
   - Bar: Presence of a bar within the restaurant premises.

3. **Temporal and Climate Conditions**:
   - Fri: Indication of whether it is Friday or not.
   - Rain: Weather condition, specifically rainy weather.

4. **Customer Attributes**:
   - Hun: Customer's hunger level.
   - Pat: Patron density, indicating how crowded the restaurant is.

5. **Service and Pricing**:
   - Price: Price range of the restaurant.
   - Res: Reservation availability for customers.

6. **Wait Time**:
   - Est: Estimated waiting time for a table.
  
7. **Decision Making**:
   - Class: Target variable indicating whether a customer decides to wait for a table ('Yes') or not ('No').


### Data Cleanup

For aesthetic purposes, we left the original tree's values as they where, however, if needed, the comments map the values to integers.

In [927]:
df1 = pd.read_csv('restaurant.csv')
df1.head()

,ID,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Class
0,X1,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,X2,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,X3,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,X4,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,X5,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No


#### Missing Values
Checking if there's any missing values.

In [928]:
print(df1.isnull().sum())

ID       0
Alt      0
Bar      0
Fri      0
Hun      0
Pat      0
Price    0
Rain     0
Res      0
Type     0
Est      0
Class    0
dtype: int64


#### Mapping Booleans to Integers

By replacing String values with small integers, it's easier for the program to process boolean values, and even the ones that have more than 2 options, for example the attribute **Price**.

After the last evaluation, we need to handle the empty (None) values in the Pat attribute. (None != 'None').

Again, if there is need to do so, just uncommenting what's below is enough.

In [929]:
'''
df1['Alt'] = df1['Alt'].replace({'No': 0, 'Yes': 1})
df1['Bar'] = df1['Bar'].replace({'No': 0, 'Yes': 1})
df1['Fri'] = df1['Fri'].replace({'No': 0, 'Yes': 1})
df1['Hun'] = df1['Hun'].replace({'No': 0, 'Yes': 1})
df1['Rain'] = df1['Rain'].replace({'No': 0, 'Yes': 1})
df1['Res'] = df1['Res'].replace({'No': 0, 'Yes': 1})
df1['Class'] = df1['Class'].replace({'No': 0, 'Yes': 1})
df1['Type'] = df1['Type'].replace({'French':0, 'Thai':1, 'Burger':2, 'Italian':3})

'''
df1['Est'] = df1['Est'].replace({'0-10': 0, '10-30': 1, '30-60': 2, '>60': 3})
df1['Price'] = df1['Price'].replace({'$':0, '$$':1,'$$$':2})

# trivial python implementation to solve the labels with empty values
def map_pat_to_int(value):
    mapping = {'None': 'None', 'Some': 'Some', 'Full': 'Full'}
    return mapping.get(value, -1)  # return -1 for empty values

df1['Pat'] = df1['Pat'].apply(map_pat_to_int)
df1 = df1.drop('ID', axis=1)
#df1.head()

### Weather <a name="weather"></a>
[[go back to the topic]](#datasets)

The 'weather' dataset holds a range of meteorological conditions and their impact on the decision to engage in an outdoor activity (specifically playing a sport).

1. **ID** - Unique instance identification
  
2. **Weather Conditions**
   - Weather: sunny, overcast, rainy
   - Temperature: temperature (F) value
   - Humidity: percentage, ranging from 0-100
   - Windy: presence or absence 

3. **Decision** - Play or not

### Data Cleanup

In [930]:
df2 = pd.read_csv('weather.csv')
df2.head()

,ID,Weather,Temp,Humidity,Windy,Play
0,1,sunny,85,85,False,no
1,2,sunny,80,90,True,no
2,3,overcast,83,86,False,yes
3,4,rainy,70,96,False,yes
4,5,rainy,68,80,False,yes


#### Missing values
Checking if there's any missing values, if so, those need to be handled accordingly.

In [931]:
print(df2.isnull().sum())

ID          0
Weather     0
Temp        0
Humidity    0
Windy       0
Play        0
dtype: int64


#### Mapping Booleans to Integers

In [932]:
df2['Windy'] = df2['Windy'].replace({False: 0, True: 1})
df2['Weather'] = df2['Weather'].replace({'sunny':0, 'overcast':1,'rainy':2})
df2['Play'] = df2['Play'].replace({'no':0, 'yes':1})
df2 = df2.drop('ID', axis=1)
df2.head()

,Weather,Temp,Humidity,Windy,Play
0,0,85,85,0,0
1,0,80,90,1,0
2,1,83,86,0,1
3,2,70,96,0,1
4,2,68,80,0,1


### Iris <a name="iris"></a>
[[go back to the topic]](#datasets)

The 'iris' dataset, which is renowned for its introdutory role in the ML world, records the morphological measurements of *Iris* flowers. The attributes recorded are the following: 

 1. **ID** - Unique instance identification
  
2. **Features**
   - Sepal Length
   - Petal Length
   - Sepal Width
   - Petal Width

3. **Decision** - Decide which species (Iris-setosa, Iris-versicolor, or Iris-virginica) the instance belongs to. 

In [933]:
df3 = pd.read_csv('iris.csv')
df3.head()

,ID,sepallength,sepalwidth,petallength,petalwidth,class
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


#### Missing Values
Checking if there's any missing values, we conclude that there are none.

In [934]:
print(df3.isnull().sum())

ID             0
sepallength    0
sepalwidth     0
petallength    0
petalwidth     0
class          0
dtype: int64


In [935]:
df3 = df3.drop('ID', axis=1)
df3.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### Creating a **scatterplot matrix**
Although not used, it's useful to check this datasets data properties.


In [936]:
sb.pairplot(df3.dropna(), hue='class')

**Note for the Iris dataset**: after careful search andevaluation, we concluded that the dataset is already clean, without outliers. For that reason, no further issues considering the csv will take place.

### Connect4 <a name="connect4"></a>
[[go back to the topic]](#datasets)

The 'connect4' csv consists of multiple instances representing endgame states of the Connect Four game. The objective is to use these configurations to predict the outcome of the game - whether it ends in a win for the first player, a win for the second player, or a draw. 

This dataset is different fundamentaly, it does not contain attributes, but only the final state and the winner.

In [937]:
df4 = pd.read_csv('connect4.csv')
df4.head()

,b,b.1,b.2,b.3,b.4,b.5,b.6,b.7,b.8,b.9,...,b.25,b.26,b.27,b.28,b.29,b.30,b.31,b.32,b.33,win
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
1,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,win
2,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
3,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
4,b,b,b,b,b,b,b,b,b,b,...,b,b,b,o,b,b,b,b,b,win


#### Missing Values
No missing values, so no need to access that problem.

In [938]:
print(df4.isnull().sum())

b       0
b.1     0
b.2     0
b.3     0
b.4     0
b.5     0
b.6     0
b.7     0
b.8     0
b.9     0
b.10    0
b.11    0
x       0
o       0
b.12    0
b.13    0
b.14    0
b.15    0
x.1     0
o.1     0
x.2     0
o.2     0
x.3     0
o.3     0
b.16    0
b.17    0
b.18    0
b.19    0
b.20    0
b.21    0
b.22    0
b.23    0
b.24    0
b.25    0
b.26    0
b.27    0
b.28    0
b.29    0
b.30    0
b.31    0
b.32    0
b.33    0
win     0
dtype: int64


# ID3 <a name="id3"></a>
[[go back to the top]](#contents)

The ID3 decision tree algorithm strategically employs Information Gain to identify optimal points for splitting the data during the tree-building process. This selection is crucial as it influences the algorithm's ability to accurately classify data.

 To quantify IG, he uses the concept of **Entropy**. By calculating Entropy, we can evaluate the effectiveness of each potential split in **reducing uncertainty**, guiding the decision tree to more effective and informed splits that **enhance classification performance**.

In [939]:
class ID3:
    def __init__(self, df):
        self.df = df
        # self.tree = self.build_tree(df) 

        self.atts = df.columns.tolist()[1:-1] 
        self.target = df.columns.tolist()[-1] 

### Initial Entropy Calculation:
- **Definition of Entropy:** Measures the amount of uncertainty in the dataset.
- **Entropy Calculation Formula:**

  $$ 
  \text{Entropy}(S) = -\sum_{i=1}^n p_i \log_2(p_i) 
  $$
  - **n:** Total number of classes in the target column.
  - **p_i:** Probability of class i, or the ratio of "number of rows with class i in the target column" to the "total number of rows" in the dataset.
  - **log_2(p_i):** Logarithm base 2 of p_i, used to calculate the entropy contribution of each class.

### Information Gain Assessment:
- **Information Gain Calculation:** Determine the gain in information by computing the change in entropy before and after partitioning the dataset based on an attribute.
- **Information Gain Formula:**


#### Information Gain Assessment:
- **Information Gain Calculation:** Determine the gain in information by computing the change in entropy before and after partitioning the dataset based on an attribute.
  - **Information Gain Formula:**
    $$
    IG(S, A) = \text{Entropy}(S)- \sum \left( \frac{|S_v|}{|S|} \cdot \text{Entropy}(S_v) \right)
  $$
    
  In this formula:
  - **S**  is the entire dataset.
  - **A** is the attribute used for partitioning.
  - **S_v**  is the size of the subset of S where the attribute A has value v.
  - **|S|**  is the total size of the dataset.
  - **Entropy(S)** is the entropy of the entire dataset S.
  - **Entropy(S_v)** is the entropy of subset S_v. 

#### Attribute Selection:
- **Optimal Attribute Choice:** Select the attribute with the maximal information gain for splitting. This is done by calculating the information gain for each attribute and comparing these values to identify the attribute that most effectively reduces uncertainty.

#### Iterative Process:
- **Repetition Until Completion:** Continuing the process iteratively, applying the steps of entropy calculation, information gain assessment, and attribute selection recursively on each subset of the dataset created by a decision node.

- The process ends when all data at a node have the same class, no remaining attributes exist for further partitioning, or another stopping criterion is met.

### Entropy <a name="entropy"></a>
[[go back to the topic]](#id3)



#### Calculating the Entropy

In order to do so, here's an implementation in python which calculates the entropy given the different values parsed as a function. 

In [940]:
import math

class ID3(ID3):
    def _entropy(self, class_counts):
        total = sum(class_counts) 
        if total == 0:
            return 0 

        entropy = 0
        for count in class_counts:
            if count > 0:
                probability = count / total
                entropy -= probability * math.log2(probability)

        return entropy

'''
class_counts = [25, 25]  
entropy = entropy(class_counts)
print(f"Entropy of the distribution is: {entropy}")
'''

'\nclass_counts = [25, 25]  \nentropy = entropy(class_counts)\nprint(f"Entropy of the distribution is: {entropy}")\n'

Now we need to parse all the class counts as a list, and compare the different entropies, to choose the one that properly divides the dataset the most.


### Calculating the Information Gain

Now, using the previous function, we need to calculate the information gain for splitting the tree using the attribute_name, and the target_name.

Next, there is an implementation in *python* which calculates the IG given the previously stated arguments.

The function starts by calculating the weighted entropy, doing the summatory described above, and then doing the formula below.

In [941]:
class ID3(ID3):
    def _calculate_entropy_of_split(self, df, target_attribute):
        """
        Calculate entropy based on the distribution of values in the target attribute.
        """
        counts = [len(df[df[target_attribute] == value]) for value in df[target_attribute].unique()]
        return self._entropy(counts)

    def _information_gain(self, df_subset, attribute_name):
        """
        Calculate the information gain for a specific attribute within a given subset of the DataFrame.
        """
        total_entropy = self._calculate_entropy_of_split(df_subset, self.target)
        weighted_entropy = 0

        for value in df_subset[attribute_name].unique():
            subset = df_subset[df_subset[attribute_name] == value]
            subset_entropy = self._calculate_entropy_of_split(subset, self.target)
            weighted_entropy += (len(subset) / len(df_subset)) * subset_entropy

        return total_entropy - weighted_entropy

### Plurality Values

According to page 660 of the manual, **if** there is need to break ties, we pick the **most common output value** among a set of examples.

Panda module has a built-in function to order by the most common, so we just need to adapt it. Although not now, it might be used for later.

In [942]:
class ID3(ID3):
    def _plurality_value(self, target_attribute):
        """Return the most common value in the target attribute."""
        return Counter(self.df[target_attribute]).most_common(1)[0][0]

### Example <a name="eg1"></a>

Let's test the new function for the Restaurant csv, splitting on the **Alt** attribute.


In [943]:
column_name = 'Alt'

unique_values = df1[column_name].unique()

class_counts = []
for arr in unique_values:
    x = df1[df1[column_name] == arr].shape[0]
    class_counts.append(x)

print(class_counts)

[6, 6]


Now we are good to go.
The entropy should be 1, as it splits the dataset evenly, but we will test the function now.

Then, we need to calculate the IG.

In [944]:
x = ID3(df1)
entr = x._entropy(class_counts)
print(f"Entropy for {class_counts} : {entr}")

Entropy for [6, 6] : 1.0


In [945]:
x = ID3(df1)

information_gains = {}

for column in x.atts:
    gain = x._information_gain(x.df, column)
    information_gains[column] = gain

# Choose the attribute with the highest information gain
max_gain_attr = max(information_gains, key=information_gains.get)

print(f"\nAttribute with the maximum information gain: {max_gain_attr} (IG: {information_gains[max_gain_attr]:.3f})")


Attribute with the maximum information gain: Pat (IG: 0.541)


The IG associated with each attribute is represented in the table below, in which we can assess that in fact, Pat has the highest IG score.

| **Attribute**  | Alt | Bar | Fri | Hun | Pat | Price | Rain | Res | Type | Est |
|------------|-----|-----|-----|-----|-----|-------|------|-----|------|-----|
| **Information Gain**  | 0.000 | 0.000 | 0.021 | 0.196 | 0.541 | 0.196 | 0.000 | 0.021 | 0.000 | 0.208 |

Now we have the base working for the ID3 algorithm, and can calculate the IG of the attributes correctly in each instance.

### Choose the best attribute

In order to do so, we just need to use the previously defined methods, to pick the index of the attribute with the highest IG. 

The ID3 method *choose_best_attribute* creates a dictionary that links all the (remaining) attributes to their respective IG, and returns the key (attribute name) of the one with the max IG.

In [946]:
class ID3(ID3):
    def choose_best_attribute(self, df_subset):        
        remaining_attributes = [col for col in df_subset.columns if col != self.target] # debug step

        information_gains = {} # dict att -> IG(att)
        for attr in remaining_attributes:
            information_gains[attr] = self._information_gain(df_subset, attr)

        return max(information_gains, key=information_gains.get)

a = ID3(df1)
subset = a.df  # In the first iteration, use the entire dataset
best_attribute = a.choose_best_attribute(subset)

print("Best Attribute for Splitting:", best_attribute)

Best Attribute for Splitting: Pat


### Restaurant Implementation

This is the simplest one. No adaptations are required, since the **values are discrete**, and not continuous values, and the **target class is binary**.

In [963]:

class ID3(ID3):
    def _build_tree(self, data):
        # debug statements from chatGPT
        from collections import Counter

        labels = data[data.columns[-1]].tolist()
        target_attribute = data.columns[-1]
        class_counts = Counter(labels)

        # Base cases
        if len(set(labels)) == 1:
            return [labels[0], class_counts[labels[0]]]  # returned 2 thing to help the debug below
        if len(data.columns) == 1:
            return [class_counts.most_common(1)[0][0], len(labels)] 

        # Choose the best attribute to split on
        best_attribute = self.choose_best_attribute(data)
        node = {best_attribute: {}}

        complete_values = self.df[best_attribute].unique()
        subset_values = data[best_attribute].unique()
        # print(f"Building tree on {best_attribute}, values: {complete_values}")
        
        # missing_values = set(complete_values) - set(subset_values)
        # print(missing_values)

        for value in complete_values:
            subset = data[data[best_attribute] == value]
            if subset.empty:
                # Assign the majority class if this value is missing in the subset
                majority_class = class_counts.most_common(1)[0][0]
                # print(f"Value '{value}' missing. Assigning majority class '{majority_class}'")
                node[best_attribute][value] = [majority_class, 0]
            else:
                # Recursively build the tree for non-empty subsets
                remaining_data = subset.drop(columns=[best_attribute])
                node[best_attribute][value] = self._build_tree(remaining_data)

        return node



In [964]:
# test the function usage 
abc = ID3(df1)
decision_tree_structure = abc._build_tree(abc.df)

print(decision_tree_structure)

{'Pat': {'Some': ['Yes', 4], 'Full': {'Hun': {'Yes': {'Type': {'French': ['No', 0], 'Thai': {'Fri': {'No': ['No', 1], 'Yes': ['Yes', 1]}}, 'Burger': ['Yes', 1], 'Italian': ['No', 1]}}, 'No': ['No', 2]}}, 'None': ['No', 2]}}


We can however define a recursive function with the corresponding indents, and make it look more like a tree. The format, although not perfect, is enough

In [977]:
class ID3(ID3):
    def print_tree(self, tree_structure):
        """
        Public function to start printing the decision tree structure from the root.
        :param tree_structure: The nested dictionary representing the decision tree.
        """
        self._recursive_print_tree(tree_structure, "")

    def _recursive_print_tree(self, tree, prefix="", indent="    "):
        # courtesy of GPT
        if isinstance(tree, dict):
            for attribute, branches in tree.items():
                print(prefix + f"{attribute}:")
                last_value = len(branches) - 1

                for index, (value, subtree) in enumerate(branches.items()):
                    is_last = index == last_value
                    value_prefix = prefix + indent
                    print(value_prefix + f"{value}:")
                    self._recursive_print_tree(subtree, value_prefix + indent, indent)

        else:
            # Leaf node handling
            leaf_value = tree[0] if isinstance(tree, list) else tree
            print(prefix + str(leaf_value))


In [978]:
teste = ID3(df1)
decision_tree_structure = abc._build_tree(abc.df)
teste.print_tree(decision_tree_structure)

Pat:
    Some:
        Yes
    Full:
        Hun:
            Yes:
                Type:
                    French:
                        No
                    Thai:
                        Fri:
                            No:
                                No
                            Yes:
                                Yes
                    Burger:
                        Yes
                    Italian:
                        No
            No:
                No
    None:
        No


#### Classify examples for Restaurant

We can create a new function, that uses tree with brackets (nested dictionary), and traverses the tree iteratively in order to classify a new example, given in a dictionary.

In [ ]:
class DecisionTree(DecisionTree):
    def classify(self, instance, node=None): # Classify a new instance using the decision tree.
        if node is None: 
            node = self.tree
        if not isinstance(node, dict):
            return node  # Leaf node with classification label
        attribute = next(iter(node))
        value = instance.get(attribute)
        next_branch = node.get(attribute).get(value, 'Unknown')
        if isinstance(next_branch, dict):
            return self.classify(instance, next_branch) # Iterate recursively to the next branch
        else:
            return next_branch

Below we can define a new instance, and given the generated tree, will take the proper decision.

In [ ]:
new_instance = {
    'Alt': 'Yes',
    'Bar': 'No',
    'Fri': 'Yes',
    'Hun': 'Yes',
    'Pat': 'Full',
    'Price': '$$$',
    'Rain': 'No',
    'Res': 'Yes',
    'Type': 'Italian',
    'Est': '10-30'
}

### Weather Implementation

Now, instead of only having discrete values, they are now continuous, in the attributes **Temperature** and **Humidity**.
